# PROYECTO SQL


# Contexto:

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Nos han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

In [4]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Antes de empezar echemos un vistazo a nuestras tablas**

In [5]:
#Primeras 5 filas de la tabla books
query = '''
SELECT *
FROM books
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
#Primeras 5 filas de la tabla authors
query = '''
SELECT *
FROM authors
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
#Primeras 5 filas de la tabla publishers
query = '''
SELECT *
FROM publishers
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
#Primeras 5 filas de la tabla ratings
query = '''
SELECT *
FROM ratings
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
#Primeras 5 filas de la tabla reviews
query = '''
SELECT *
FROM reviews
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Objetivo del Estudio

**Conocer el mercado**

¿Cuántos libros se han publicado después del 1 de enero del 2000?

¿Cuántas reseñas han hecho los usuarios y la calificación promedio para cada libro?

¿Cuál ha sido la editorial que ha publicado más libros (> a 50 páginas)?

¿Cuál es el mejor autor, el que tiene la más alta calificación promedio del libro (> 50 páginas)?

El número promedio de reseñas entre usuarios que calificaron más de 50 libros.

## Encontremos el número de libros publicados después del 1 de enero de 2000.

In [10]:
#Veamos el número de libros publicacdos después del 1 de enero del 2020
query = '''
SELECT 
COUNT (book_id) AS total_libros
FROM books
WHERE 
publication_date > '2000-01-01';
'''
pd.io.sql.read_sql(query, con = engine)

,total_libros
0,819


In [11]:
#Veamos la fecha más reciente
query = '''
SELECT 
MAX(publication_date) AS Fecha_Max
FROM books;
'''
pd.io.sql.read_sql(query, con = engine)

,fecha_max
0,2020-03-31


**Se han escrito 819 libros desde el 1 de Enero del 2000 hasta el 31 de Marzo del 2020**


## Encontremos el número de reseñas de usuarios y la calificación promedio para cada libro.

In [12]:
#Número de reseñas de usuarios
query = '''
SELECT 
COUNT(DISTINCT review_id) AS Total_reviews
FROM reviews;
'''
pd.io.sql.read_sql(query, con = engine)

,total_reviews
0,2793


In [13]:
#Número de reseñas para cada libro y calificación promedio para cada libro
query = '''
SELECT 
books.title AS titulo,
ROUND(AVG(ratings.rating),2) AS RATING_PROMEDIO,
COUNT(reviews.review_id) AS total_reviews
FROM books
LEFT JOIN ratings ON ratings.book_id = 
books.book_id
LEFT JOIN reviews ON reviews.book_id =
books.book_id
GROUP BY
books.title
ORDER BY
RATING_PROMEDIO DESC;
'''
pd.io.sql.read_sql(query, con = engine)

,titulo,rating_promedio,total_reviews
0,Captivating: Unveiling the Mystery of a Woman'...,5.00,4
1,Evening Class,5.00,4
2,In the Hand of the Goddess (Song of the Liones...,5.00,6
3,The Big Bad Wolf (Alex Cross #9),5.00,4
4,A Dirty Job (Grim Reaper #1),5.00,16
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,2.25,12
995,His Excellency: George Washington,2.00,4
996,Junky,2.00,4
997,Drowning Ruth,2.00,9


**Hemos creado una vista donde podemos ver cuáles libros tienen el mejor rating y podemos ver cuántas reviews tienen pero, para tener datos más limpios deberíamos filtrar un poco más los datos (por número de páginas de los libros, cuántas reviews han recibido, por ejemplo)**


## Identifiquemos la editorial que ha publicado el mayor número de libros con más de 50 páginas

In [14]:
#Editoriales con mayor número de libros publicados
query = '''
SELECT 
publishers.publisher AS editorial,
publishers.publisher_id AS id_editorial,
COUNT(books.book_id) AS total_libros
FROM publishers
LEFT JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
books.num_pages > 50
GROUP BY publishers.publisher, publishers.publisher_id
ORDER BY
total_libros DESC
LIMIT 3;
'''
pd.io.sql.read_sql(query, con = engine)

,editorial,id_editorial,total_libros
0,Penguin Books,212,42
1,Vintage,309,31
2,Grand Central Publishing,116,25


**La editorial con más libros publicados es "Penguin Books" con 42 libros, seguida de la editorial "Vintage" con 309**

## Identifiquemos al autor que tiene la más alta calificación promedio del libro: veamos solo los libros con al menos 50 calificaciones.

In [15]:
#Autor con mayor calificación promedio 
query = '''
SELECT
books.book_id AS libro_id,
books.title AS titulo,
authors.author_id AS autor_id,
authors.author AS autor,
ROUND(AVG(ratings.rating),2) AS rating_promedio
FROM
books 
INNER JOIN (
SELECT
book_id
FROM
ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
) AS libros_filtrados ON libros_filtrados.book_id = books.book_id
INNER JOIN ratings ON ratings.book_id = books.book_id
INNER JOIN authors ON authors.author_id = books.author_id
WHERE books.num_pages > 50
GROUP BY authors.author_id, authors.author, books.book_id, books.title
ORDER BY rating_promedio DESC
LIMIT 3;
'''
pd.io.sql.read_sql(query, con = engine)

,libro_id,titulo,autor_id,autor,rating_promedio
0,302,Harry Potter and the Prisoner of Azkaban (Harr...,236,J.K. Rowling/Mary GrandPré,4.41
1,722,The Fellowship of the Ring (The Lord of the Ri...,240,J.R.R. Tolkien,4.39
2,299,Harry Potter and the Chamber of Secrets (Harry...,236,J.K. Rowling/Mary GrandPré,4.29


**Los autores con la calificación más alta promedio es J.K. Rowling/Mary GrandPré, tomando los libros con al menos 50 calificaciones.**

**Estos libros podrían entrar en una sección de recomendados ya que tienen un buen rating y fueron evaluados por muchas personas**

## Encontremos el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [16]:
#Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

query = '''
SELECT 
ROUND(AVG(review_count), 2) AS promedio_reviews
FROM (
    SELECT 
        username,
        COUNT(review_id) AS review_count
    FROM reviews
    LEFT JOIN books ON books.book_id = reviews.book_id
    WHERE books.num_pages > 50
    GROUP BY username
    HAVING COUNT(DISTINCT reviews.book_id) > 50
) AS usuarios_filtrados;
'''
pd.io.sql.read_sql(query, con = engine)

,promedio_reviews
0,None


**Vaya!! Parece que no tenemos usuarios que hayan hecho reseñas de texto y leido más de 50 libros ("libros" que tengan más de 50 páginas)**

In [17]:
#Veamos relajando los parámetros de los filtros, quitando el número de páginas mínimo y a partir de más de 10 libros
#Número promedio de reseñas de texto entre los usuarios que calificaron más de 10 libros

query = '''
SELECT 
ROUND(AVG(review_count), 2) AS promedio_reviews
FROM (
    SELECT 
        username,
        COUNT(review_id) AS review_count
    FROM reviews
    LEFT JOIN books ON books.book_id = reviews.book_id
    GROUP BY username
    HAVING COUNT(DISTINCT reviews.book_id) > 10
) AS usuarios_filtrados;
'''
pd.io.sql.read_sql(query, con = engine)

,promedio_reviews
0,17.74


**Para los usuarios que han leido más de 10 libros(sin importar de cuántas páginas sean) el promedio de reviews con texto es de 17.74, parece que no se hacen muchas reviews de los libros leídos, podríamos fomentar las reviews en nuestra página o aplicación**